In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib_venn import venn2

# Define parameter mapping between HOLAPS and Fluxnet
holaps_to_fluxnet = {
    'GHF': 'G_F_MDS',
    'H': ['H_CORR', 'H_F_MDS'],
    'LE': ['LE_CORR', 'LE_F_MDS'],
    'Rn': 'NETRAD',
    'Ta': 'TA_F',
    'Td': 'TS_F_MDS_1',
    'rain': 'P_F',
    'sm1': 'SWC_F_MDS_1'
}

# File paths
fluxnet_file = r'C:\Deepak\Comparison\comprehensive_selected_periods_analysis.xlsx'
holaps_file = r'C:\Deepak\Comparison\comprehensive_holaps_analysis.xlsx'
output_directory = r'C:\Deepak\Comparison\comparison_analysis5'

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

def load_and_prepare_data():
    """Load and prepare data from both HOLAPS and Fluxnet"""
    
    # Load Fluxnet data
    print("Loading Fluxnet data...")
    fluxnet_df = pd.read_excel(fluxnet_file, sheet_name='All_Selected_Periods_Data')
    
    # Load HOLAPS data
    print("Loading HOLAPS data...")
    holaps_df = pd.read_excel(holaps_file, sheet_name='All_Data')
    
    # Clean station names
    fluxnet_df['station_clean'] = fluxnet_df['station'].str.replace('progressive_trend_tables_', '')
    holaps_df['station_clean'] = holaps_df['station'].str.replace('holaps_trend_tables_', '')
    
    # Get all 44 HOLAPS stations
    all_holaps_stations = set(holaps_df['station_clean'].unique())
    fluxnet_stations = set(fluxnet_df['station_clean'].unique())
    
    print(f"All HOLAPS stations: {len(all_holaps_stations)}")
    print(f"Fluxnet stations: {len(fluxnet_stations)}")
    print(f"HOLAPS stations: {sorted(all_holaps_stations)}")
    
    return fluxnet_df, holaps_df, all_holaps_stations, fluxnet_stations

def create_individual_comparisons(fluxnet_df, holaps_df, all_holaps_stations, fluxnet_stations):
    """Create individual comparisons for all parameter pairs"""
    
    comparison_results = []
    station_details = []
    
    for season in ['yearly', 'JJA']:
        print(f"\nProcessing {season} season...")
        
        # Filter data by season
        fluxnet_season = fluxnet_df[fluxnet_df['season'] == season]
        holaps_season = holaps_df[holaps_df['season'] == season]
        
        for holaps_param, fluxnet_params in holaps_to_fluxnet.items():
            # Handle multiple mappings
            if isinstance(fluxnet_params, list):
                for fluxnet_param in fluxnet_params:
                    process_individual_comparison(
                        holaps_season, fluxnet_season, 
                        holaps_param, fluxnet_param, 
                        season, comparison_results, station_details, 
                        all_holaps_stations, fluxnet_stations
                    )
            else:
                process_individual_comparison(
                    holaps_season, fluxnet_season, 
                    holaps_param, fluxnet_params, 
                    season, comparison_results, station_details,
                    all_holaps_stations, fluxnet_stations
                )
    
    return pd.DataFrame(comparison_results), pd.DataFrame(station_details)

def process_individual_comparison(holaps_data, fluxnet_data, holaps_param, fluxnet_param, season, results, station_details, all_holaps_stations, fluxnet_stations):
    """Process individual comparison for one parameter pair"""
    
    # Get HOLAPS data for all 44 stations
    holaps_param_data = holaps_data[
        (holaps_data['parameter'] == holaps_param) & 
        (holaps_data['station_clean'].isin(all_holaps_stations))
    ]
    
    # Get Fluxnet data
    fluxnet_param_data = fluxnet_data[fluxnet_data['parameter'] == fluxnet_param]
    
    # Get stations with data
    holaps_stations_with_data = set(holaps_param_data['station_clean'].unique())
    fluxnet_stations_with_data = set(fluxnet_param_data['station_clean'].unique())
    
    # Initialize counters
    both_sig_stations = []
    holaps_sig_only_stations = []
    fluxnet_sig_only_stations = []
    both_insig_stations = []
    holaps_only_stations = []  # HOLAPS stations not in Fluxnet
    fluxnet_only_stations = []  # Fluxnet stations not in HOLAPS
    
    both_sig = 0
    holaps_sig_only = 0
    fluxnet_sig_only = 0
    both_insig = 0
    holaps_only_count = 0
    fluxnet_only_count = 0
    
    # Process each HOLAPS station
    for station in all_holaps_stations:
        holaps_station_data = holaps_param_data[holaps_param_data['station_clean'] == station]
        
        if len(holaps_station_data) > 0:
            holaps_sig_level = holaps_station_data['significance'].iloc[0]
            holaps_sig = holaps_sig_level in ['high', 'medium']
            holaps_pval = holaps_station_data['p_value'].iloc[0]
            holaps_slope = holaps_station_data['slope'].iloc[0]
            
            # Check if station exists in Fluxnet
            if station in fluxnet_stations_with_data:
                fluxnet_station_data = fluxnet_param_data[fluxnet_param_data['station_clean'] == station]
                
                if len(fluxnet_station_data) > 0:
                    fluxnet_sig_level = fluxnet_station_data['significance'].iloc[0]
                    fluxnet_sig = fluxnet_sig_level in ['high', 'medium']
                    fluxnet_pval = fluxnet_station_data['p_value'].iloc[0]
                    fluxnet_slope = fluxnet_station_data['slope'].iloc[0]
                    
                    # Store station details
                    station_details.append({
                        'season': season,
                        'holaps_parameter': holaps_param,
                        'fluxnet_parameter': fluxnet_param,
                        'station': station,
                        'dataset_presence': 'both',
                        'holaps_significance': holaps_sig_level,
                        'fluxnet_significance': fluxnet_sig_level,
                        'holaps_p_value': holaps_pval,
                        'fluxnet_p_value': fluxnet_pval,
                        'agreement': holaps_sig == fluxnet_sig,
                        'agreement_type': 'both_significant' if holaps_sig and fluxnet_sig else 
                                         'both_insignificant' if not holaps_sig and not fluxnet_sig else 
                                         'disagreement'
                    })
                    
                    if holaps_sig and fluxnet_sig:
                        both_sig += 1
                        both_sig_stations.append(station)
                    elif holaps_sig and not fluxnet_sig:
                        holaps_sig_only += 1
                        holaps_sig_only_stations.append(station)
                    elif not holaps_sig and fluxnet_sig:
                        fluxnet_sig_only += 1
                        fluxnet_sig_only_stations.append(station)
                    else:
                        both_insig += 1
                        both_insig_stations.append(station)
            else:
                # Station only in HOLAPS
                holaps_only_count += 1
                holaps_only_stations.append(station)
                
                station_details.append({
                    'season': season,
                    'holaps_parameter': holaps_param,
                    'fluxnet_parameter': fluxnet_param,
                    'station': station,
                    'dataset_presence': 'holaps_only',
                    'holaps_significance': holaps_sig_level,
                    'fluxnet_significance': 'N/A',
                    'holaps_p_value': holaps_pval,
                    'fluxnet_p_value': 'N/A',
                    'agreement': 'N/A',
                    'agreement_type': 'holaps_only'
                })
    
    # Count Fluxnet-only stations
    for station in fluxnet_stations_with_data:
        if station not in all_holaps_stations:
            fluxnet_only_count += 1
            fluxnet_only_stations.append(station)
    
    # Calculate statistics
    total_comparable = both_sig + holaps_sig_only + fluxnet_sig_only + both_insig
    agreement_count = both_sig + both_insig
    agreement_rate = (agreement_count / total_comparable * 100) if total_comparable > 0 else 0
    
    results.append({
        'season': season,
        'holaps_parameter': holaps_param,
        'fluxnet_parameter': fluxnet_param,
        'total_holaps_stations': len(all_holaps_stations),
        'total_comparable_stations': total_comparable,
        'holaps_only_stations': holaps_only_count,
        'fluxnet_only_stations': fluxnet_only_count,
        
        # Significant counts
        'both_significant': both_sig,
        'holaps_significant_only': holaps_sig_only,
        'fluxnet_significant_only': fluxnet_sig_only,
        'both_insignificant': both_insig,
        
        # Statistics
        'agreement_rate_percent': agreement_rate,
        'both_sig_percent': (both_sig / total_comparable * 100) if total_comparable > 0 else 0,
        'both_insig_percent': (both_insig / total_comparable * 100) if total_comparable > 0 else 0,
        
        # Station lists
        'both_sig_stations': ', '.join(sorted(both_sig_stations)),
        'holaps_sig_only_stations': ', '.join(sorted(holaps_sig_only_stations)),
        'fluxnet_sig_only_stations': ', '.join(sorted(fluxnet_sig_only_stations)),
        'both_insig_stations': ', '.join(sorted(both_insig_stations)),
        'holaps_only_stations': ', '.join(sorted(holaps_only_stations)),
        'fluxnet_only_stations': ', '.join(sorted(fluxnet_only_stations))
    })

def create_individual_venn_diagrams(comparison_df, all_holaps_stations):
    """Create individual Venn diagrams for each parameter pair"""
    
    for season in ['yearly', 'JJA']:
        print(f"\nCreating individual Venn diagrams for {season} season...")
        
        season_data = comparison_df[comparison_df['season'] == season]
        
        for _, row in season_data.iterrows():
            create_single_venn_diagram(row, season, all_holaps_stations)

def create_single_venn_diagram(comparison_row, season, all_holaps_stations):
    """Create a single Venn diagram for one parameter pair"""
    
    holaps_param = comparison_row['holaps_parameter']
    fluxnet_param = comparison_row['fluxnet_parameter']
    
    # Create figure
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    
    # Get counts for Venn diagram
    both_sig = comparison_row['both_significant']
    holaps_sig_only = comparison_row['holaps_significant_only']
    fluxnet_sig_only = comparison_row['fluxnet_significant_only']
    both_insig = comparison_row['both_insignificant']
    
    total_comparable = comparison_row['total_comparable_stations']
    holaps_only = comparison_row['holaps_only_stations']
    fluxnet_only = comparison_row['fluxnet_only_stations']
    
    # Create Venn diagram for significant stations
    if total_comparable > 0:
        # Use only the comparable stations for the Venn diagram
        venn2(subsets=(holaps_sig_only, fluxnet_sig_only, both_sig),
              set_labels=(f'{holaps_param}\n(HOLAPS)', f'{fluxnet_param}\n(Fluxnet)'), 
              ax=ax)
    else:
        ax.text(0.5, 0.5, 'No comparable stations', ha='center', va='center', 
                transform=ax.transAxes, fontsize=12, fontweight='bold')
    
    # Add comprehensive statistics
    stats_text = (
        f'Total HOLAPS Stations: {len(all_holaps_stations)}\n'
        f'Comparable Stations: {total_comparable}\n'
        f'HOLAPS-only Stations: {holaps_only}\n'
        f'Fluxnet-only Stations: {fluxnet_only}\n'
        f'Both Significant: {both_sig}\n'
        f'HOLAPS Only Significant: {holaps_sig_only}\n'
        f'Fluxnet Only Significant: {fluxnet_sig_only}\n'
        f'Both Insignificant: {both_insig}\n'
        f'Agreement Rate: {comparison_row["agreement_rate_percent"]:.1f}%'
    )
    
    ax.text(0.5, -0.3, stats_text, transform=ax.transAxes, fontsize=10,
            ha='center', va='top', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
            linespacing=1.5)
    
    ax.set_title(f'{holaps_param} (HOLAPS) vs {fluxnet_param} (Fluxnet)\n{season} Season', 
                 fontsize=14, fontweight='bold', pad=20)
    
    # Adjust layout and save
    plt.tight_layout(rect=[0, 0, 1, 0.85])
    filename = f'{holaps_param}_vs_{fluxnet_param}_{season}.png'
    plt.savefig(os.path.join(output_directory, filename), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  Created Venn diagram: {filename}")

def create_comprehensive_summary(comparison_df, station_details_df, all_holaps_stations):
    """Create comprehensive summary report"""
    
    with pd.ExcelWriter(os.path.join(output_directory, 'individual_comparison_summary.xlsx')) as writer:
        
        # Overall summary
        summary_data = []
        for season in ['yearly', 'JJA']:
            season_data = comparison_df[comparison_df['season'] == season]
            
            for _, row in season_data.iterrows():
                summary_data.append({
                    'Season': season,
                    'HOLAPS_Parameter': row['holaps_parameter'],
                    'Fluxnet_Parameter': row['fluxnet_parameter'],
                    'Total_HOLAPS_Stations': row['total_holaps_stations'],
                    'Comparable_Stations': row['total_comparable_stations'],
                    'HOLAPS_Only_Stations': row['holaps_only_stations'],
                    'Fluxnet_Only_Stations': row['fluxnet_only_stations'],
                    'Both_Significant': row['both_significant'],
                    'HOLAPS_Only_Significant': row['holaps_significant_only'],
                    'Fluxnet_Only_Significant': row['fluxnet_significant_only'],
                    'Both_Insignificant': row['both_insignificant'],
                    'Agreement_Rate': f"{row['agreement_rate_percent']:.1f}%",
                    'Both_Sig_Percent': f"{row['both_sig_percent']:.1f}%",
                    'Both_Insig_Percent': f"{row['both_insig_percent']:.1f}%"
                })
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Comparison_Summary', index=False)
        
        # Station details
        station_details_df.to_excel(writer, sheet_name='Station_Details', index=False)
        
        # All HOLAPS stations list
        stations_df = pd.DataFrame({'All_HOLAPS_Stations': sorted(all_holaps_stations)})
        stations_df.to_excel(writer, sheet_name='HOLAPS_Stations', index=False)
        
        # Best agreements
        best_agreements = comparison_df.nlargest(10, 'agreement_rate_percent')[
            ['season', 'holaps_parameter', 'fluxnet_parameter', 'agreement_rate_percent', 
             'both_sig_percent', 'both_insig_percent', 'total_comparable_stations']
        ]
        best_agreements.to_excel(writer, sheet_name='Best_Agreements', index=False)

def main():
    """Main function to run the individual comparison analysis"""
    
    print("Starting Individual HOLAPS-Fluxnet Comparison for All 44 Stations...")
    
    # Load data
    fluxnet_df, holaps_df, all_holaps_stations, fluxnet_stations = load_and_prepare_data()
    
    # Create individual comparisons
    comparison_df, station_details_df = create_individual_comparisons(
        fluxnet_df, holaps_df, all_holaps_stations, fluxnet_stations
    )
    
    # Create individual Venn diagrams
    create_individual_venn_diagrams(comparison_df, all_holaps_stations)
    
    # Create comprehensive summary
    create_comprehensive_summary(comparison_df, station_details_df, all_holaps_stations)
    
    print(f"\nIndividual comparison analysis completed!")
    print(f"Results saved to: {output_directory}")
    print(f"Main summary file: individual_comparison_summary.xlsx")
    print(f"Total Venn diagrams created: {len(comparison_df)}")
    print(f"All 44 HOLAPS stations included in analysis")

if __name__ == "__main__":
    main()

Starting Individual HOLAPS-Fluxnet Comparison for All 44 Stations...
Loading Fluxnet data...
Loading HOLAPS data...
All HOLAPS stations: 44
Fluxnet stations: 51
HOLAPS stations: ['AT-Neu', 'BE-Bra', 'BE-Lon', 'BE-Vie', 'CH-Cha', 'CH-Dav', 'CH-Fru', 'CZ-BK1', 'CZ-wet', 'DE-Geb', 'DE-Gri', 'DE-Hai', 'DE-HoH', 'DE-Hzd', 'DE-Kli', 'DE-Lnf', 'DE-Obe', 'DE-RuR', 'DE-RuS', 'DE-Tha', 'DK-Sor', 'ES-Agu', 'ES-LJu', 'FR-Aur', 'FR-Bil', 'FR-Gri', 'FR-Hes', 'FR-LBr', 'FR-Lam', 'IT-BCi', 'IT-Col', 'IT-Cp2', 'IT-Cpz', 'IT-Lav', 'IT-MBo', 'IT-Ren', 'IT-Ro2', 'IT-SR2', 'IT-SRo', 'IT-Tor', 'IT-TrF', 'NL-Loo', 'RU-Fyo', 'SE-Htm']

Processing yearly season...

Processing JJA season...

Creating individual Venn diagrams for yearly season...


C:\Users\chinthap\AppData\Local\Temp\ipykernel_9480\1635638182.py:285: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout(rect=[0, 0, 1, 0.85])


  Created Venn diagram: GHF_vs_G_F_MDS_yearly.png
  Created Venn diagram: H_vs_H_CORR_yearly.png
  Created Venn diagram: H_vs_H_F_MDS_yearly.png
  Created Venn diagram: LE_vs_LE_CORR_yearly.png
  Created Venn diagram: LE_vs_LE_F_MDS_yearly.png
  Created Venn diagram: Rn_vs_NETRAD_yearly.png
  Created Venn diagram: Ta_vs_TA_F_yearly.png
  Created Venn diagram: Td_vs_TS_F_MDS_1_yearly.png
  Created Venn diagram: rain_vs_P_F_yearly.png
  Created Venn diagram: sm1_vs_SWC_F_MDS_1_yearly.png

Creating individual Venn diagrams for JJA season...
  Created Venn diagram: GHF_vs_G_F_MDS_JJA.png
  Created Venn diagram: H_vs_H_CORR_JJA.png
  Created Venn diagram: H_vs_H_F_MDS_JJA.png
  Created Venn diagram: LE_vs_LE_CORR_JJA.png
  Created Venn diagram: LE_vs_LE_F_MDS_JJA.png
  Created Venn diagram: Rn_vs_NETRAD_JJA.png
  Created Venn diagram: Ta_vs_TA_F_JJA.png
  Created Venn diagram: Td_vs_TS_F_MDS_1_JJA.png
  Created Venn diagram: rain_vs_P_F_JJA.png
  Created Venn diagram: sm1_vs_SWC_F_MDS_1_JJA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib_venn import venn3


# --- Configuration ---
fluxnet_file = r'C:\Deepak\Comparison\comprehensive_selected_periods_analysis.xlsx'
holaps_file = r'C:\Deepak\Comparison\comprehensive_holaps_analysis.xlsx'
era5_file = r'C:\Deepak\Comparison\comprehensive_era5_analysis.xlsx'
output_dir = r'C:\Deepak\Comparison\separate_plots_agreement_final'
os.makedirs(output_dir, exist_ok=True)


# --- Parameter Mappings ---
# First 8 plots: MDS versions
param_list_mds = [
    {'holaps': 'GHF', 'fluxnet': 'G_F_MDS', 'era5': 'GHF', 'label': 'GHF'},
    {'holaps': 'H', 'fluxnet': 'H_F_MDS', 'era5': 'H', 'label': 'H_MDS'},
    {'holaps': 'LE', 'fluxnet': 'LE_F_MDS', 'era5': 'LE', 'label': 'LE_MDS'},
    {'holaps': 'Rn', 'fluxnet': 'NETRAD', 'era5': 'Rn', 'label': 'Rn'},
    {'holaps': 'Ta', 'fluxnet': 'TA_F', 'era5': 'Ta', 'label': 'Ta'},
    {'holaps': 'Td', 'fluxnet': 'TS_F_MDS_1', 'era5': 'Td', 'label': 'Td'},
    {'holaps': 'rain', 'fluxnet': 'P_F', 'era5': 'rain', 'label': 'rain'},
    {'holaps': 'sm1', 'fluxnet': 'SWC_F_MDS_1', 'era5': 'sm1', 'label': 'sm1'},
]


# Last 2 plots: CORR versions
param_list_corr = [
    {'holaps': 'H', 'fluxnet': 'H_CORR', 'era5': 'H', 'label': 'H_CORR'},
    {'holaps': 'LE', 'fluxnet': 'LE_CORR', 'era5': 'LE', 'label': 'LE_CORR'},
]


def load_data():
    """Load and prepare data"""
    fluxnet_df = pd.read_excel(fluxnet_file, sheet_name='All_Selected_Periods_Data')
    holaps_df = pd.read_excel(holaps_file, sheet_name='All_Data')
    era5_df = pd.read_excel(era5_file, sheet_name='All_Data')
    
    for df in [fluxnet_df, holaps_df, era5_df]:
        df['station_clean'] = df['station'].str.replace('(progressive|holaps|era5)_trend_tables_', '', regex=True)
    
    all_stations = set(holaps_df['station_clean'].unique())
    return fluxnet_df, holaps_df, era5_df, all_stations


def get_stats(data, station, param):
    """Get slope sign and significance"""
    station_data = data[data['station_clean'] == station]
    if station_data.empty: 
        return 0, False, False
        
    param_data = station_data[station_data['parameter'] == param]
    if param_data.empty: 
        return 0, False, False
    
    if pd.notna(param_data['slope'].iloc[0]):
        slope = param_data['slope'].iloc[0]
        slope_sign = 1 if slope > 0 else -1
        
        sig = False
        if 'p_value' in param_data.columns and pd.notna(param_data['p_value'].iloc[0]):
            sig = param_data['p_value'].iloc[0] < 0.2
        elif 'significant' in param_data.columns and pd.notna(param_data['significant'].iloc[0]):
            sig = bool(param_data['significant'].iloc[0])
            
        return slope_sign, sig, True
    return 0, False, False


def analyze_3way(flux_df, holaps_df, era5_df, stations, f_param, h_param, e_param):
    """Analyze 3-way agreement"""
    f_sig_set = set()
    h_sig_set = set()
    e_sig_set = set()
    
    agree_FH = 0
    agree_FE = 0
    agree_HE = 0
    common_count = 0
    
    for st in stations:
        f_sign, f_sig, f_has = get_stats(flux_df, st, f_param)
        h_sign, h_sig, h_has = get_stats(holaps_df, st, h_param)
        e_sign, e_sig, e_has = get_stats(era5_df, st, e_param)
        
        if f_has and h_has and e_has:
            common_count += 1
            
            if f_sig: 
                f_sig_set.add(st)
            if h_sig: 
                h_sig_set.add(st)
            if e_sig: 
                e_sig_set.add(st)
            
            # Check agreement: Both must be Significant AND have Same Sign
            if f_sig and h_sig and f_sign == h_sign:
                agree_FH += 1
            if f_sig and e_sig and f_sign == e_sign:
                agree_FE += 1
            if h_sig and e_sig and h_sign == e_sign:
                agree_HE += 1
    
    if common_count == 0:
        return None
                
    return {
        'venn_sets': (f_sig_set, h_sig_set, e_sig_set),
        'common': common_count,
        'agree_FH': agree_FH,
        'agree_FE': agree_FE,
        'agree_HE': agree_HE,
        'f_sig': len(f_sig_set),
        'h_sig': len(h_sig_set),
        'e_sig': len(e_sig_set)
    }


def create_plot_file_2x2(results, season, part_name, titles, figsize=(16, 18)):
    """Generate 2x2 plot with stats BELOW each Venn diagram - ALL CYAN - TIGHT SPACING"""
    if not results or len(results) == 0:
        return

    fig = plt.figure(figsize=figsize)
    
    # Create grid with TIGHT spacing between venn and stats
    gs = fig.add_gridspec(4, 2, left=0.08, right=0.95, 
                          top=0.96, bottom=0.04, 
                          height_ratios=[3.5, 0.9, 3.5, 0.9],  # REDUCED stats height
                          hspace=0.15, wspace=0.35)  # REDUCED hspace for tight spacing
    
    # Create axes: (venn1, stats1, venn2, stats2) for top row, repeat for bottom
    axes_pairs = [
        # Top-left: venn and stats
        (fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[1, 0])),
        # Top-right: venn and stats
        (fig.add_subplot(gs[0, 1]), fig.add_subplot(gs[1, 1])),
        # Bottom-left: venn and stats
        (fig.add_subplot(gs[2, 0]), fig.add_subplot(gs[3, 0])),
        # Bottom-right: venn and stats
        (fig.add_subplot(gs[2, 1]), fig.add_subplot(gs[3, 1])),
    ]
    
    for idx, ((ax_venn, ax_stats), res, title) in enumerate(zip(axes_pairs, results, titles)):
        if res is None:
            ax_venn.text(0.5, 0.5, 'No Common Data', ha='center', va='center')
            ax_venn.set_title(title, fontsize=11, fontweight='bold')
            ax_stats.axis('off')
            continue
        
        # Plot Venn3
        try:
            v = venn3(res['venn_sets'], set_labels=('Fluxnet', 'Holaps', 'ERA5'), 
                     ax=ax_venn, alpha=0.5)
        except Exception:
            pass

        ax_venn.set_title(title, fontsize=12, fontweight='bold', pad=10)
        
        # Find best agreement
        rates = {
            'F-H': res['agree_FH'],
            'F-E': res['agree_FE'],
            'H-E': res['agree_HE']
        }
        best_pair = max(rates, key=rates.get)
        
        # Check if all three values are the same
        all_same = (res['agree_FH'] == res['agree_FE'] == res['agree_HE'])
        
        # Build text for stats panel - COMPACT
        text_parts = [
            f"Cmn: {res['common']} | Sig: F={res['f_sig']} H={res['h_sig']} E={res['e_sig']} | ",
        ]
        
        # Build agreement lines - SINGLE LINE
        fh_line = f"F-H: {res['agree_FH']}"
        fe_line = f"F-E: {res['agree_FE']}"
        he_line = f"H-E: {res['agree_HE']}"
        
        if all_same:
            text_parts.append(f"{fh_line}  {fe_line}  {he_line}")
        else:
            if best_pair == 'F-H':
                text_parts.append(f"► {fh_line}  {fe_line}  {he_line}")
            elif best_pair == 'F-E':
                text_parts.append(f"{fh_line}  ► {fe_line}  {he_line}")
            else:  # H-E
                text_parts.append(f"{fh_line}  {fe_line}  ► {he_line}")
        
        box_text = "\n".join(text_parts)
        
        # Add stats panel below venn - ALL CYAN COLOR - TIGHT
        ax_stats.axis('off')
        ax_stats.text(0.5, 0.5, box_text, 
                     transform=ax_stats.transAxes, ha='center', va='center',
                     bbox=dict(boxstyle='round,pad=0.4', facecolor='lightcyan', alpha=0.95,
                              edgecolor='darkblue', linewidth=1.2),
                     fontsize=8, fontfamily='monospace', color='black')
    
    filename = f"venn_{season}_{part_name}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Created: {filename}")


def create_plot_file_1x2(results, season, part_name, titles, figsize=(14, 12)):
    """Generate 1x2 plot with stats BELOW each Venn diagram - ALL CYAN - TIGHT SPACING"""
    if not results or len(results) == 0:
        return

    fig = plt.figure(figsize=figsize)
    
    # Create grid with TIGHT spacing
    gs = fig.add_gridspec(2, 2, left=0.08, right=0.95, 
                          top=0.96, bottom=0.04, 
                          height_ratios=[3.5, 0.9],  # REDUCED stats height
                          hspace=0.15, wspace=0.35)  # REDUCED hspace
    
    # Create axes: (venn1, venn2) top row, (stats1, stats2) bottom row
    axes_venn = [fig.add_subplot(gs[0, i]) for i in range(2)]
    axes_stats = [fig.add_subplot(gs[1, i]) for i in range(2)]
    
    for idx, (ax_venn, ax_stats, res, title) in enumerate(zip(axes_venn, axes_stats, results, titles)):
        if res is None:
            ax_venn.text(0.5, 0.5, 'No Common Data', ha='center', va='center')
            ax_venn.set_title(title, fontsize=11, fontweight='bold')
            ax_stats.axis('off')
            continue
        
        # Plot Venn3
        try:
            v = venn3(res['venn_sets'], set_labels=('Fluxnet', 'Holaps', 'ERA5'), 
                     ax=ax_venn, alpha=0.5)
        except Exception:
            pass

        ax_venn.set_title(title, fontsize=12, fontweight='bold', pad=10)
        
        # Find best agreement
        rates = {
            'F-H': res['agree_FH'],
            'F-E': res['agree_FE'],
            'H-E': res['agree_HE']
        }
        best_pair = max(rates, key=rates.get)
        
        # Check if all three values are the same
        all_same = (res['agree_FH'] == res['agree_FE'] == res['agree_HE'])
        
        # Build text for stats panel - COMPACT
        text_parts = [
            f"Cmn: {res['common']} | Sig: F={res['f_sig']} H={res['h_sig']} E={res['e_sig']} | ",
        ]
        
        # Build agreement lines - SINGLE LINE
        fh_line = f"F-H: {res['agree_FH']}"
        fe_line = f"F-E: {res['agree_FE']}"
        he_line = f"H-E: {res['agree_HE']}"
        
        if all_same:
            text_parts.append(f"{fh_line}  {fe_line}  {he_line}")
        else:
            if best_pair == 'F-H':
                text_parts.append(f"► {fh_line}  {fe_line}  {he_line}")
            elif best_pair == 'F-E':
                text_parts.append(f"{fh_line}  ► {fe_line}  {he_line}")
            else:  # H-E
                text_parts.append(f"{fh_line}  {fe_line}  ► {he_line}")
        
        box_text = "\n".join(text_parts)
        
        # Add stats panel below venn - ALL CYAN COLOR - TIGHT
        ax_stats.axis('off')
        ax_stats.text(0.5, 0.5, box_text, 
                     transform=ax_stats.transAxes, ha='center', va='center',
                     bbox=dict(boxstyle='round,pad=0.4', facecolor='lightcyan', alpha=0.95,
                              edgecolor='darkblue', linewidth=1.2),
                     fontsize=8, fontfamily='monospace', color='black')
    
    filename = f"venn_{season}_{part_name}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Created: {filename}")


def main():
    print("Loading data...")
    f_df, h_df, e_df, stations = load_data()
    print(f"Total stations: {len(stations)}\n")
    
    for season in ['yearly', 'JJA']:
        print(f"=== Processing {season} ===")
        
        f_seas = f_df[f_df['season'] == season]
        h_seas = h_df[h_df['season'] == season]
        e_seas = e_df[e_df['season'] == season]
        
        results_mds = []
        titles_mds = []
        
        # Process first 8 MDS parameters
        for param_map in param_list_mds:
            h_param = param_map['holaps']
            f_param = param_map['fluxnet']
            e_param = param_map['era5']
            label = param_map['label']
            
            result = analyze_3way(f_seas, h_seas, e_seas, stations, f_param, h_param, e_param)
            results_mds.append(result)
            titles_mds.append(label)
        
        # Part 1: First 4 MDS (GHF, H_MDS, LE_MDS, Rn)
        create_plot_file_2x2(results_mds[0:4], season, "part1_4plots", 
                            titles_mds[0:4], figsize=(16, 18))
        
        # Part 2: Next 4 MDS (Ta, Td, rain, sm1)
        create_plot_file_2x2(results_mds[4:8], season, "part2_4plots", 
                            titles_mds[4:8], figsize=(16, 18))
        
        # Part 3: Last 2 CORR (H_CORR, LE_CORR)
        results_corr = []
        titles_corr = []
        
        for param_map in param_list_corr:
            h_param = param_map['holaps']
            f_param = param_map['fluxnet']
            e_param = param_map['era5']
            label = param_map['label']
            
            result = analyze_3way(f_seas, h_seas, e_seas, stations, f_param, h_param, e_param)
            results_corr.append(result)
            titles_corr.append(label)
        
        create_plot_file_1x2(results_corr, season, "part3_2plots",
                            titles_corr, figsize=(14, 12))
    
    print("\n✓ Analysis complete!")


if __name__ == "__main__":
    main()
